在进行数据分析和建模的过程中，**大量的时间花在了数据准备上**： 加载，清理，转换和重新排列，这样的工作占用了分析师的80%的时间

有时数据存储在文件或数据库中的方式**对于特定的任务来说格式并不正确**，许多研究人员选择使用通用编程语言（Python,Perl,R,或者Java)或Unix文本处理工具进行从一种形式到另一种形式的转换； **pandas和Python内置的语言公共为你提供了一个高级，灵活和快速的工具集，使你能够将数据处理为正确的形式**

`pandas`中的大部分设计实现都是由真实世界的应用需求所驱动的

In [1]:
import pandas as pd
from pandas import Series, DataFrame # 最常用的类

In [2]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True) # numpy 不要使用科学计数法，小数4位

## 处理缺失值

缺失数据在很多数据分析应用中出现，**pandas的目标之一就是尽可能无痛的处理缺失值**， **`pandas`对象的所有描述性统计信息默认情况下都是排除缺失值的**

**`pandas`对象中表现缺失值的方式并不完美，但是它对大部分用户来说是有用的**

- 对于**数值型数据**，pandas使用**浮点值`NaN`**(`Not a Number`)表示缺失值，我们称`NaN`为**容易检测到的缺失值**
- 在`pandas`中，我们采用R语言中的编程惯例，**将缺失值成为`NA`，`not avaliable` 不可用**

在统计学中，`NA`数据可以是**不存在的数据**或者**存在但不可观察的数据**
- 例如数据在收集过程中出现了问题
- 当清洗数据用于分析时，对缺失数据本身进行分析以确定数据收集问题或数据丢失导致的数据偏差都非常重要

Python内建的`None`值在对象数组中也被当做`NA`处理

In [3]:
type(np.nan)

float

In [6]:
np.NAN is np.nan

True

In [7]:
type(np.NaN)

float

In [10]:
pd.NA?

In [11]:
type(pd.NA)

pandas._libs.missing.NAType

In [12]:
string_data = Series( ['aardvard', 'artichoke', np.nan, 'avocado'] )
string_data

0     aardvard
1    artichoke
2          NaN
3      avocado
dtype: object

In [13]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [18]:
string_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In [21]:
string_data.isnull??

isnull
- Return a boolean same-sized object indicating if the values are NA. NA values, such as None or :attr:`numpy.NaN`, gets mapped to True
values.Everything else gets mapped to False values. Characters such as emptystrings ``''`` or :attr:`numpy.inf` are not considered NA values(unless you set ``pandas.options.mode.use_inf_as_na = True``).

In [24]:
pd.isnull??

isna
- Return a boolean same-sized object indicating if the values are NA.NA values, such as None or :attr:`numpy.NaN`, gets mapped to True
values.Everything else gets mapped to False values. Characters such as emptystrings ``''`` or :attr:`numpy.inf` are not considered NA values
(unless you set ``pandas.options.mode.use_inf_as_na = True``).

In [27]:
pd.isnull??

![isnull是isna的签名](https://gitee.com/zhao_long/image-store/raw/master/img/20201114090852.png)

In [28]:
string_data[0] = None
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [29]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

In [31]:
string_data[1]= pd.NA
string_data

0       None
1       <NA>
2        NaN
3    avocado
dtype: object

In [32]:
string_data.isnull()

0     True
1     True
2     True
3    False
dtype: bool

In [33]:
datas = Series(np.arange(6.))
datas

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
5    5.0
dtype: float64

In [34]:
datas[0] = np.nan
datas

0    NaN
1    1.0
2    2.0
3    3.0
4    4.0
5    5.0
dtype: float64

In [36]:
datas[1] = pd.NA
datas

0     NaN
1    <NA>
2       2
3       3
4       4
5       5
dtype: object

In [37]:
pd.NA?

###  过滤缺失值

有很多过滤缺失值的方法
1. `pandas.isnull`和`布尔索引`手动过滤缺失值
2. `dropna`过滤缺失值
    - `Series`上使用，**返回Series中所有非空数据集及其索引**
    - `DataFrame`上稍微复杂，**涉及轴**， 默认情况下，**删除包含缺失值的行**
    - `How=all`:所有值均为`NA`的行或列
    - `thresh`： **这一行除去NA值，剩余数值的数量(观察值的数量)大于等于n，便显示这一行** `Require that many non-NA values`

`dropna` 根据**每个标签的值**是否是缺失来**筛选轴标签**，并根据允许丢失的数据量来确定阈值

In [39]:
data = Series([1, np.nan, 3.5, np.nan, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [40]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [42]:
data[~data.isnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [43]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [45]:
data = [ [1., 6.5, 3], [1., np.nan, np.nan], [np.nan,np.nan,np.nan], [np.nan,6.5,3.] ]
data = DataFrame(data)
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [61]:
data.dropna?

In [47]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [48]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [49]:
data[3] = np.nan
data

,0,1,2,3
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [51]:
data.dropna(1).shape

(4, 0)

In [53]:
data.dropna(1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [54]:
df = DataFrame(np.random.randn(7,3))
df.iloc[:4, 1] = np.nan 
df.iloc[:2, 2] = np.nan 
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [55]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [57]:
df.dropna(thresh=1)

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [58]:
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [59]:
df.dropna(thresh=3)

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [60]:
df.dropna(thresh=4)

,0,1,2


### 补全缺失值

**可能需要补全缺失值，而不是过滤缺失值，或丢弃缺失值**

大多数情况下，**主要使用`fillna`方法补全缺失值**

参数 | 描述
- | -
value | 标量值或者字典型对象 用于填充缺失值
method | 插值方法  默认`ffill`
asix | 需要插值的轴 默认axis=0
inplace | 修改被调用的对象，而不是生成一个新对象
limit | 用于前向或后向填写时最大的填充范围

**使用字典时，为不同的列设值不同的值**

In [62]:
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [63]:
df.fillna(100)

,0,1,2
0,-0.204708,100.000000,100.000000
1,-0.555730,100.000000,100.000000
2,0.092908,100.000000,0.769023
3,1.246435,100.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [64]:
df.fillna?

In [66]:
df.fillna( {1 : 100, 2: 200})

,0,1,2
0,-0.204708,100.000000,200.000000
1,-0.555730,100.000000,200.000000
2,0.092908,100.000000,0.769023
3,1.246435,100.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [67]:
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [70]:
df.fillna(100, inplace=True)
df

,0,1,2
0,-0.204708,100.000000,100.000000
1,-0.555730,100.000000,100.000000
2,0.092908,100.000000,0.769023
3,1.246435,100.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [72]:
df = DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = np.nan 
df.iloc[4:, 2] = np.nan 
df

,0,1,2
0,-1.265934,0.119827,-1.063512
1,0.332883,-2.359419,-0.199543
2,-1.541996,NaN,-1.307030
3,0.286350,NaN,-0.753887
4,0.331286,NaN,NaN
5,0.246674,NaN,NaN


In [75]:
df.fillna(method='ffill')

,0,1,2
0,-1.265934,0.119827,-1.063512
1,0.332883,-2.359419,-0.199543
2,-1.541996,-2.359419,-1.307030
3,0.286350,-2.359419,-0.753887
4,0.331286,-2.359419,-0.753887
5,0.246674,-2.359419,-0.753887


In [76]:
df.fillna(method='ffill', axis=1)

,0,1,2
0,-1.265934,0.119827,-1.063512
1,0.332883,-2.359419,-0.199543
2,-1.541996,-1.541996,-1.307030
3,0.286350,0.286350,-0.753887
4,0.331286,0.331286,0.331286
5,0.246674,0.246674,0.246674


In [77]:
df.fillna(method='ffill', axis=1, limit=1)

,0,1,2
0,-1.265934,0.119827,-1.063512
1,0.332883,-2.359419,-0.199543
2,-1.541996,-1.541996,-1.307030
3,0.286350,0.286350,-0.753887
4,0.331286,0.331286,NaN
5,0.246674,0.246674,NaN


In [78]:
df.fillna(method='bfill', axis=1)

,0,1,2
0,-1.265934,0.119827,-1.063512
1,0.332883,-2.359419,-0.199543
2,-1.541996,-1.307030,-1.307030
3,0.286350,-0.753887,-0.753887
4,0.331286,NaN,NaN
5,0.246674,NaN,NaN


In [84]:
df.fillna(df.max(1))

,0,1,2
0,-1.265934,0.119827,-1.063512
1,0.332883,-2.359419,-0.199543
2,-1.541996,0.332883,-1.307030
3,0.286350,0.332883,-0.753887
4,0.331286,0.332883,-1.307030
5,0.246674,0.332883,-1.307030


## 数据转换

### 删除重复值

`DataFrame`中会出现**重复的行**，
- **`duplicated`用于检测重复的行，返回一个布尔值的Series**，反映的是每一行是否和前面的行重复
- **`drop_duolicated`用于删除`duplicated`返回的为`True`的行**
- `keep`参数： 默认保留第一个重复的行， `last`保留最后的行
- `subset` 所有列参与统计重复，也可以指定列的组合，传入**列名的列表**

In [85]:
data = DataFrame( {
    'k1': ['one', 'two'] * 3 + ['two'],
    'k2': [1,1,2,3,3,4,4]
} )
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [89]:
data.duplicated?

- subset : column label or sequence of labels, optional
    Only consider certain columns for identifying duplicates, by
    default use all of the columns.

In [87]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [88]:
data.duplicated(['k1'])

0    False
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool

In [90]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [92]:
data['v1']= range(7)

In [93]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [94]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool

In [95]:
data.duplicated(['k1'])

0    False
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool

In [100]:
data.duplicated(['k1'], keep='last')

0     True
1     True
2     True
3     True
4    False
5     True
6    False
dtype: bool

In [99]:
data.drop_duplicates(['k1'], keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


In [98]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


### 使用函数或映射进行数据转换

对于许多数据集，希望**基于`DataFrame`中的数组，列或列中的数值进行一些转换**

`Series`的`map`方法**接受一个函数或一个包含映射关系的一个字典对象**

In [101]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [102]:
meat_to_animal = {
    'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [104]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [105]:
lowercased.map(meat_to_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [106]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [107]:
data['food'].map( lambda x : meat_to_animal[x.lower()] )

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使用`map`是一种可以便捷的按元素转换及其他清洗操作的方法

### 替代值

使用`fillna`填充缺失值是通用值替换的特殊案例，`map`可以用来修改一个对象中的子集信息

`replace`提供了更为简单灵活的实现
- 生成新的Series
- 参数： `to_place, value` 替换一个值
- 参数：`to_place`： 列表，把列表中的值替换
- 参数： `[to_place1, to_place2] [value1, value2]`
- 参数： {`to_place`: vallue, `to_place`: value}

In [108]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [110]:
data.replace??

In [111]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [112]:
data.replace(  {-999: np.nan, -1000: 1000}) 

0       1.0
1       NaN
2       2.0
3       NaN
4    1000.0
5       3.0
dtype: float64

### 重命名轴索引

和Series的值一样，**可以通过函数或某种形式的映射对轴标签进行类似的转换**，生成新的且带有不同标签的对象

In [114]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


1. 轴索引的map方法

In [116]:
data.index.map( lambda x : x[:4].upper() )

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [117]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [118]:
data.index = data.index.map( lambda x : x[:4].upper() )

In [119]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


2. rename方法： **创建数据集转换后的版本，且不修改原有的数据集， 而且不是index的方法**
    - 可以结合字典使用，为轴标签提供新的值 

In [122]:
data.rename(
    index = str.title, 
    columns=str.upper
)


,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [125]:
data.rename(
    index = {'OHIO': 'INAIANT'}, 
    columns={'three': '3'}
)


,one,two,3,four
INAIANT,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 离散化和分箱

**连续值经常需要离散化，或者分离成“箱子”进行分析**

In [126]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [127]:
bins = [18, 25,35,60,100]
bins

[18, 25, 35, 60, 100]

In [128]:
cats = pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [129]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [130]:
cats.describe

<bound method Categorical.describe of [(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]>

In [131]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [133]:
type(cats)

pandas.core.arrays.categorical.Categorical

In [137]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

`pandas.cut`计算出的**“箱”** ， 把一个数组按照给定的数据进行离散，分箱，返回的是一个`Categorical`对象,可以**当做一个表示箱名的字符串数组**
- 内部包含一个`categories` **类别数组**， 指定不同的类别名称
- `code`属性**数据标签**
- `pd.value_counts`是**对箱数量的计数**
- `right=True`: 默认值，表示**箱子的区间右边的闭合的**
- `lablels`: 传递一个列表或数组来传入**自定义的箱名**
- 如果**传入整数个的箱代替显示的箱边**，pandas**会根据数据中的最大最小值计算出等长的箱**

In [134]:
pd.cut?

In [136]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [138]:
pd.cut(ages, bins, right=False, labels=['Youth', 'YoungAdult', 'MiddleAged', 'Senior'])

['Youth', 'Youth', 'YoungAdult', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [140]:
pd.cut(ages, bins, right=False, labels=['Youth', 'YoungAdult', 'MiddleAged', 'Senior']).categories

Index(['Youth', 'YoungAdult', 'MiddleAged', 'Senior'], dtype='object')

In [142]:
data = np.random.rand(20)
data

array([0.8524, 0.9499, 0.1073, 0.9107, 0.3361, 0.8264, 0.8981, 0.0427,
       0.1958, 0.2945, 0.627 , 0.0862, 0.1429, 0.5158, 0.6893, 0.8566,
       0.6474, 0.5816, 0.7111, 0.2524])

In [143]:
pd.cut(data, 4, precision=2)

[(0.72, 0.95], (0.72, 0.95], (0.042, 0.27], (0.72, 0.95], (0.27, 0.5], ..., (0.72, 0.95], (0.5, 0.72], (0.5, 0.72], (0.5, 0.72], (0.042, 0.27]]
Length: 20
Categories (4, interval[float64]): [(0.042, 0.27] < (0.27, 0.5] < (0.5, 0.72] < (0.72, 0.95]]

In [144]:
pd.value_counts(pd.cut(data, 4, precision=2))

(0.72, 0.95]     6
(0.5, 0.72]      6
(0.042, 0.27]    6
(0.27, 0.5]      2
dtype: int64

`qcut`是一个与分箱密切先关的函数，**基于样本分位数进行分箱**； 取决于数据的分布
- 使用`cut`通常**不会使每个箱具有相同数据量的数据点**
- 由于`qcut`使用**样本的分位数，可以获取等长的箱**
- 参数不再是数组，而是**需要分箱的数字或者自定义的分位数(0-1之间的数字)**

In [146]:
cats = pd.qcut( np.random.randn(10000), 4)

In [147]:
cats

[(-0.696, -0.0134], (0.664, 3.928], (-0.0134, 0.664], (0.664, 3.928], (-0.696, -0.0134], ..., (-3.746, -0.696], (-3.746, -0.696], (-3.746, -0.696], (-0.0134, 0.664], (-0.0134, 0.664]]
Length: 10000
Categories (4, interval[float64]): [(-3.746, -0.696] < (-0.696, -0.0134] < (-0.0134, 0.664] < (0.664, 3.928]]

In [148]:
pd.value_counts(cats)

(0.664, 3.928]       2500
(-0.0134, 0.664]     2500
(-0.696, -0.0134]    2500
(-3.746, -0.696]     2500
dtype: int64

In [151]:
cats = pd.qcut( np.random.randn(10000), [0, 0.1, 0.5, 0.9 , 1])

In [152]:
pd.value_counts(cats)

(-0.00844, 1.299]     4000
(-1.295, -0.00844]    4000
(1.299, 3.962]        1000
(-3.509, -1.295]      1000
dtype: int64

In [153]:
cats

[(-0.00844, 1.299], (-1.295, -0.00844], (-1.295, -0.00844], (-0.00844, 1.299], (-3.509, -1.295], ..., (-1.295, -0.00844], (-0.00844, 1.299], (-1.295, -0.00844], (-1.295, -0.00844], (-0.00844, 1.299]]
Length: 10000
Categories (4, interval[float64]): [(-3.509, -1.295] < (-1.295, -0.00844] < (-0.00844, 1.299] < (1.299, 3.962]]

### 检测和过滤异常值

过滤或转换异常值在很大程序上是应用数组操作的事情

In [154]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.015556,0.004018,-0.032238,0.015686
std,0.981998,1.000297,0.993869,1.021294
min,-3.646441,-3.148164,-4.150859,-3.113411
25%,-0.677777,-0.681336,-0.710321,-0.643932
50%,0.030453,0.008133,-0.004698,-0.006657
75%,0.659243,0.633584,0.609302,0.664315
max,3.127275,2.710026,2.914424,3.038583


In [156]:
col = data[2]
col[ np.abs(col) > 3]

160   -3.172614
332   -4.150859
Name: 2, dtype: float64

使用any或者and函数

In [160]:
(np.abs(data) > 3).any() 

0    True
1    True
2    True
3    True
dtype: bool

In [162]:
data[ (np.abs(data) > 3).any(1) ]

,0,1,2,3
160,0.287872,-1.568923,-3.172614,-0.782276
188,1.562889,-3.025204,0.956712,0.938825
272,-3.646441,-1.210580,-0.629924,0.352066
332,0.563140,-0.740642,-4.150859,-0.054803
354,1.130201,-3.148164,-0.264340,-1.158593
684,1.722857,0.389649,0.274137,-3.113411
780,0.979510,-0.910844,-0.807588,3.018700
923,3.127275,0.744660,-1.166108,-1.305069
991,0.149161,-1.994717,-1.180563,3.038583


In [163]:
(np.abs(data) > 3).any(1)

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [164]:
np.sign(data)

,0,1,2,3
0,-1.0,1.0,1.0,1.0
1,-1.0,1.0,-1.0,-1.0
2,-1.0,1.0,1.0,1.0
3,1.0,1.0,-1.0,-1.0
4,-1.0,1.0,-1.0,-1.0
...,...,...,...,...
995,-1.0,-1.0,1.0,-1.0
996,-1.0,1.0,-1.0,-1.0
997,-1.0,-1.0,1.0,1.0
998,1.0,1.0,-1.0,-1.0


In [167]:
data[ (np.abs(data) > 3).any(1) ] = np.sign(data)*3

In [171]:
data.iloc[272]

0   -1.0
1   -1.0
2   -1.0
3    1.0
Name: 272, dtype: float64

In [168]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.016680,0.010483,-0.027097,0.013751
std,0.970626,0.990165,0.982294,1.009639
min,-2.653639,-2.792740,-2.751373,-2.990578
25%,-0.677777,-0.681336,-0.712215,-0.648091
50%,0.030453,0.008133,-0.004698,-0.006657
75%,0.665820,0.635765,0.612977,0.668206
max,2.855379,2.710026,2.914424,2.961172


### 置换和随机抽样

`np.random.permutation` **根据你想要的轴长度产生一个表示新顺序的整数数组**

In [173]:
df = DataFrame(np.arange(5 * 4).reshape(5,4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [174]:
sampler = np.random.permutation(5)
sampler

array([2, 3, 0, 1, 4])

In [175]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19


整数数据可以用于**基于iloc的索引**或**等价的`take`函数**中

In [176]:
df.iloc[sampler]

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19


选出一个不含有替代值的随机子集，可以使用`sample`方法,有替代值的话(重复值)，`replace=True`

In [178]:
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
2,8,9,10,11
3,12,13,14,15


In [180]:
df.sample(n=8, replace=True)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11
2,8,9,10,11
1,4,5,6,7
2,8,9,10,11
1,4,5,6,7


### 计算指标/虚拟变量

**将分类变量转换为“虚拟”或“指标”矩阵**是另一种用于统计建模或机器学习的转换操作

**如果DataFrame中的一列有k的不同的值，则可以衍生出一个k列的值为1和0的矩阵或DataFrame**， pandas有一个函数`get_dummies`用于实现该功能

In [187]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1':list('ABCDEF')})
df

,key,data1
0,b,A
1,b,B
2,a,C
3,c,D
4,a,E
5,b,F


In [188]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [189]:
pd.get_dummies(df['key'], prefix='key')

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [192]:
df['num'] = range(6)

In [193]:
df

,key,data1,num
0,b,A,0
1,b,B,1
2,a,C,2
3,c,D,3
4,a,E,4
5,b,F,5


In [194]:
pd.get_dummies(df, prefix='key')

,num,key_a,key_b,key_c,key_A,key_B,key_C,key_D,key_E,key_F
0,0,0,1,0,1,0,0,0,0,0
1,1,0,1,0,0,1,0,0,0,0
2,2,1,0,0,0,0,1,0,0,0
3,3,0,0,1,0,0,0,1,0,0
4,4,1,0,0,0,0,0,0,1,0
5,5,0,1,0,0,0,0,0,0,1


In [195]:
pd.get_dummies(df['num'], prefix='key')

,key_0,key_1,key_2,key_3,key_4,key_5
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,0,0,0,1,0,0
4,0,0,0,0,1,0
5,0,0,0,0,0,1


In [196]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})

In [198]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [201]:
df[['data1']].join(dummies)

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [202]:
df['data1']

0    0
1    1
2    2
3    3
4    4
5    5
Name: data1, dtype: int64

In [207]:
df[ ['data1','key'] ]

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [208]:
mnames = ['movie_id', 'title', 'genres']

In [213]:
movies = pd.read_csv('pydata-book/datasets/movielens/movies.dat',sep='::' ,header=None, names=mnames)
movies.head()

<ipython-input-213-2a7a7b7f16a5>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('pydata-book/datasets/movielens/movies.dat',sep='::' ,header=None, names=mnames)


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [217]:
all_genres = []

for x  in movies.genres:
    all_genres.extend(x.split('|'))

In [218]:
geners = pd.unique(all_genres)
len(geners)

18

In [219]:
geners

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [221]:
len(movies)

3883

In [222]:
np.zeros?

![虚拟变量](https://gitee.com/zhao_long/image-store/raw/master/img/20201114162139.png)

In [224]:
zero_matrix = np.zeros((len(movies), len(geners)))

In [225]:
dummies = DataFrame(zero_matrix, columns=geners )

In [226]:
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [227]:
gen = movies.genres[0]

In [228]:
gen

"Animation|Children's|Comedy"

In [229]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [230]:
dummies.columns.get_indexer(gen.split('|')) # 获取索引

array([0, 1, 2])

In [232]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[ i, indices ] = 1

In [233]:
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [234]:
movie_windc = movies.join(dummies.add_prefix('Genre_'))

In [237]:
movie_windc.iloc[:4]

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
movie_windc.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                               1
Genre_Children's                              1
                               ...             
Genre_War                                     0
Genre_Musical                                 0
Genre_Mystery                                 0
Genre_Film-Noir                               0
Genre_Western                                 0
Name: 0, Length: 21, dtype: object

In [238]:
np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]


In [240]:
pd.cut(values, bins)

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [239]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 字符串操作

由于Python在字符串和文本操作上的便利性，是Python称为一个流行的原生数据集操作语言已经有很长时间了

字符串对象的内奸方法使得大部分文本操作非常简单，。对于更复杂的模式匹配和文本操作，正则表达式可能是需要的

**`Pandas`允许你将字符串和正则表达式简洁应用到整个数据数组上，此外还能处理数据缺失带来的困扰**

### 字符串对象方法

内建的字符串方法是足够处理的

### 正则表达式

正则表达式**提供了一种在文本中灵活查找或匹配字符串模式的方法**

单个表达式称为`regex`,是根据正则表达式语言形成的字符串

Python内建的`re`模块是用于将正则表达式应用到字符串上的库

### pandas中的向量化字符串函数

清理杂乱的数据集用于分析通常需要大量的字符串处理和正则化，但是**包含字符串的列有时含有缺失数据，使事情变得更加复杂**

`map`可以将函数应用到每个值上，但是遇到NA/null值会失败

**`Series`有面向数组的方法用于跳过NA值的字符串操作**，这些方法**通过`Series`的`str`属性进行调用**

**正则表达式也可以结合任意的re模块进行调用**

In [241]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [ ]:
data.map(str)

有多重方法进行向量化的元素检索，比如`str.get`或在`str`属性内部检索

In [244]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [247]:
import re
data1 = data.str.findall(pattern, flags=re.IGNORECASE)

In [248]:
data1

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [261]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [262]:
data1.str.get(1)

nan

In [271]:
data1.str[1]


Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64